# Twitter Sentiment Analysis

## Notebook 3 : Deep Learning comparison of model ANN, LSTM and BERT

In [1]:
# new library download

!pip install torch==2.0.0 --quiet
!pip install pytorch_lightning==2.2.0 --quiet
!pip install torchtext==0.15.1 --quiet
!pip install mlflow==2.9.0 --quiet
!pip install gensim==4.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# Importing libraires for file and time operations
import os
import time
import warnings
warnings.filterwarnings('ignore')

# Libraries for random number generation
import random


# Basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline


from itertools import product

# library for experiment tracking
import mlflow

# Library for splitting data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# libraries for Word2Vec
import gensim
from gensim.models import Word2Vec

# libraries for nltk
import nltk
from nltk import tokenize
from nltk.tokenize import WhitespaceTokenizer

# Librarie from torchtext for tokenization
import torchtext
from torchtext.data.utils import get_tokenizer

# Libraries for Deep Learning with PyTorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Library for clearing Jupyter Notebook cell output
from IPython.display import clear_output

# Libraries for model evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Library for clearing Jupyter Notebook cell output
from IPython.display import clear_output

In [4]:
# new library importing inbetween




# Seeting seeds for reproducibility


In [5]:
def set_seed(seed_value):
  # set python's random seed
  random.seed(seed_value)

  # set numpy's random seed
  np.random.seed(seed_value)

  os.environ['PYTHONHASHSEED'] = str(seed_value)
  # set Pytorch;s CPU seed
  torch.manual_seed(seed_value)
  # Set CuDNN deterministic mode (for further reproducibility)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  # Set PyTorch's GPU seed (if available)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed(42)

## Loading dataset

In [6]:
# copy files from google drive to google colab content

!cp '/content/drive/MyDrive/Twitter_Sentiment_Analysis_1/training_processed_data.csv' '/content/'
!cp '/content/drive/MyDrive/Twitter_Sentiment_Analysis_1/SKIP_Word2Vec.model' '/content/'
!cp '/content/drive/MyDrive/Twitter_Sentiment_Analysis_1/CBOW_Word2Vec.model' '/content/'

In [7]:
# Load the dataset

dataset = pd.read_csv('training_processed_data.csv',nrows = 500000)
dataset.sample(5)

,target,text
104241,0,good sore head stuffy nose symptom swine flu
199676,0,work work
140199,0,mimunm year life lodger bastard got life throw...
132814,0,sorry hear hang ok
408697,0,i got fired job


In [8]:
dataset.dropna(axis = 0, how = "any", inplace = True)
dataset.isnull().sum()

,0
target,0
text,0


In [9]:
dataset.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492950 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   target  492950 non-null  int64 
 1   text    492950 non-null  object
dtypes: int64(1), object(1)
memory usage: 50.8 MB


## Splitting the data into Train and Test data

In [10]:
# Split the dataset into Training and Testing data
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['target'], test_size = 0.25, random_state = 42)




## Data Vectorization

In [11]:
# Define the path to the pre-trained Word2Vec models
cbow_path = '/content/CBOW_Word2Vec.model'
sg_path = '/content/SKIP_Word2Vec.model'

# Load the models in the memory
cbow = Word2Vec.load(cbow_path)
sg = Word2Vec.load(sg_path)

converting the raw data into vectorize format with 300 dimensions using the above mentioned models

In [12]:
def vectorizer(text, word2vec_model):
  x = len(text)
  y = 300

  matrix = np.zeros((x,y))

  # iterate through each text sample in the dataframe
  for i in range(x):
    words = WhitespaceTokenizer().tokenize(text.iloc[i])

    for word in words:
      word_vec = []
      if word in word2vec_model.wv:
        word_vec.append(word2vec_model.wv.get_vector(word))

      if word_vec:
        matrix[i] = np.mean(word_vec, axis=0)

  return matrix

Let's vectorize the data using CBOW_Word2Vec model

In [13]:
# Start the data vectorization using CBOW
start_time = time.time()

matrix_train_cbow = vectorizer(X_train,cbow)
matrix_test_cbow = vectorizer(X_test,cbow)

print(f" Data Vectorization completed using cbow. Duration {time.time()-start_time} secs")

 Data Vectorization completed using cbow. Duration 79.0372850894928 secs


In [14]:
print(matrix_train_cbow.shape)

(369712, 300)


In [15]:
print(matrix_train_cbow[1].shape)

(300,)


In [16]:
# Start the data vectorization with Skipgram
start_time = time.time()

matrix_train_sg = vectorizer(X_train,sg)
matrix_test_sg = vectorizer(X_test,sg)

print(f"Data Vectorization completed using Skipgram. Duration {time.time()- start_time}")


Data Vectorization completed using Skipgram. Duration 70.24965858459473


## Setting the computational device

In [17]:
# Choose GPU if availabel, otherwise use CPU
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(f"Device being used is {device}")

Device being used is cuda


## First:- Basic Neural Network

Classifier class sets up the network layer, while train and test functions manage the training and evaluation process.

In [18]:
# Define neural network classifier
class classifier(nn.Module):
  def __init__(self, input_size, hidden_size, out_size):
    super(classifier, self).__init__()

    # Define hidden layer with ReLU activation
    self.features = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU())

    # Define output Layer
    self.out = nn.Linear(hidden_size, out_size)

  #Define foreard pass
  def forward(self, X):
    feature = self.features(X)
    output = self.out(feature)

    return output

In [19]:
# Training Function for a single epoch

def train(train_loader, net, epoch, criterion, optimizer):

  # set the network to training mode
  net.train()

  # Initialize list to store batch loss
  epoch_loss = []

  # Iterate through each batch in the training data loader
  for batch in train_loader:
    data, target = batch

    # Move the data and labels to the chosen device
    data , target = data.to(device), target.to(device)

    # Forward pass: commute predictions and loss
    pred = net(data)
    loss = criterion(pred.squeeze(),target.float())
    epoch_loss.append(loss.cpu().data)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Convert epoch loss to numpy array and compute mean
  epoch_loss = np.asarray(epoch_loss)

  return epoch_loss.mean()

In [20]:
# Testing Function for single epoch

def test(test_loader, net, epoch, criterion):
  # set the network for evaluation mode
  net.eval()

  # Disable gradient calculation for performance calculation
  with torch.no_grad():
    # Initiate loss list to store the loss
    epoch_loss = []

    # Iterate through each batch in the test data loader
    for batch in test_loader:
      data,target = batch

      # Move data and target to the chosen device
      data, target = data.to(device), target.to(device)

      # Forward pass: compute predictoins and loss
      pred = net(data)
      loss = criterion(pred.squeeze(), target.float())
      epoch_loss.append(loss.cpu().data)

    #Convcer the epoch loss to numpy array and compute the mean
    epoch_loss = np.asarray(epoch_loss)

    return epoch_loss.mean()


In [21]:
from tqdm import tqdm

In [31]:
# Main function to train the neural network

def train_nn(matrix_train, matrix_test, args, hidden_size = 32, input_size = 300, output_size = 1):

  # start MLflow run for experiment tracking
  with mlflow.start_run(experiment_id = experiment_id):
    # log hyperparameters
    for key, value in args.items():
      mlflow.log_param(key,value)

    # Log neural netword architechture details
    mlflow.log_param('input_size', input_size)
    mlflow.log_param('hidden_size', hidden_size)
    mlflow.log_param('output_size', output_size)



    # converting training and testing matrices to pytorch tensors
    X_train = torch.from_numpy(matrix_train).float()
    Y_train = torch.from_numpy(y_train.values)
    train_data = TensorDataset(X_train,Y_train)

    X_test = torch.from_numpy(matrix_test).float()
    Y_test = torch.from_numpy(y_test.values)
    test_data = TensorDataset(X_test, Y_test)

    # Create data Loaders for training and testing datasets
    train_loader = DataLoader(train_data, batch_size = args['batch_size'], shuffle = True,
                              num_workers = args['num_workers'])
    test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = True,
                            num_workers = args['num_workers'])

    # Initialize the neular network
    net = classifier(input_size, hidden_size, output_size)
    net.to(device)

    #Define the loss function
    criterion = nn.BCEWithLogitsLoss().to(device)

    # Define optimizer depending on params
    if args['optimizer'] == 'SGD':
            optimizer = optim.SGD(net.parameters(), lr=args['lr'])

    if args['optimizer'] == 'Adam':
      optimizer = optim.Adam(net.parameters(), lr = args['lr'],
                            weight_decay = args['weight_decay'])

    # Initializes variable to keep track of losses and other metrics
    train_losses, test_losses = [], []
    best_test_loss = float('inf')
    best_model_epoch = -1
    no_improvement_epoch = 0

    # Records the time at which training starts
    start_time = time.time()

    # Main Training of neural network
    for epoch in tqdm(range(args['num_epochs'])):

      #Clear the output and prints the currnet training status
      # clear_output(wait = True)

      print("Training neural network ... Epoch " + str(epoch) + '/' +str(args['num_epochs']-1))
      if best_model_epoch != -1:
        print("Best test loss epoch: "+ str(best_model_epoch))

      # Train the network and logs the loss
      epoch_train_loss = train(train_loader, net, epoch, criterion, optimizer)
      train_losses.append(epoch_train_loss)

      # Test the network and logs the loss
      epoch_test_loss = test(test_loader, net, epoch, criterion)
      test_losses.append(epoch_test_loss)

      # Checks for improvement in test loss
      diff = best_test_loss - epoch_test_loss
      if epoch_test_loss< best_test_loss:
        best_test_loss = epoch_test_loss
        #Save the state of the best model for future reference
        torch.save(net.state_dict(), 'backup_best_model.pth')
        best_model_epoch = epoch

      # Implement early stopping based on predefined tolerance and patience
      if diff> args['tolerance']:
        no_improvement_epochs = 0
      else:
        no_improvement_epochs +=1
      if no_improvement_epochs >= args['patience']:
        print("Early Stopping at epoch: ", epoch)
        break

    # Record the duration for which the model training took
    duration = time.time()-start_time

    # Clear the output and prints the final training status
    # clear_output(wait = True)

    print(f"Neural network training has finished. Elasped time {duration :.2f}")
    print("Best model saved")
    print("Best test loss epoch: "+ str(best_model_epoch))

    # Load the best model
    true_test_labels, predicted_test_labels = [], []
    net.load_state_dict(torch.load("backup_best_model.pth"))

    # Use the best model to make predictions on the train data
    with torch.no_grad():
      for batch in test_loader:
        data, target = batch
        data, target = data.to(device), target.to(device)
        pred = net(data)
        true_test_labels.extend(target.cpu().numpy())
        predicted_test_labels.extend((torch.sigmoid(pred).squeeze()>0.5).cpu().numpy())

    # Calculate performance metrics
    test_accuracy = accuracy_score(true_test_labels, predicted_test_labels)
    test_precision = precision_score(true_test_labels, predicted_test_labels)
    test_recall = recall_score(true_test_labels, predicted_test_labels)
    test_f1 = f1_score(true_test_labels, predicted_test_labels)

    # print the performance metrics of the best model
    print("Best model metrics: ")
    print(f"Accuracy: {test_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1 {test_f1}")


  return (train_losses, test_losses, best_model_epoch)

## Setting up Mlflow Experiment Tracking

In [23]:
# experiment name for Mlflow
experiment_name = 'BASIC_NN'

# check if experiment exist, if not create
if mlflow.get_experiment_by_name(experiment_name) is None:
  experiment_id = mlflow.create_experiment(experiment_name)
else:
  experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id


## Parameter Grid Definition

In [34]:
# define parameter grid for hyperparameter tuning

param_grid = {
    'vectorization' :['cbow','sg'],
    'criterion': ['BCEWithLogitsLoss'],
    'optimizer' :['Adam'],
    'batch_size' : [500],
    'num_workers':[2],
    'lr' : [0.0001,0.0005,0.001],
    'weight_decay' : [0.0001,0.001],
    'num_epochs':[500],
    'tolerance' : [0.0003],
    'patience' : [25]
}

# generate all combinations of hyperparameters for grid search
all_combinations = [dict(zip(param_grid,v)) for v in product(*param_grid.values())]

In [35]:
# remove already trained combination from experiment tracking list
def remove_trained(combinations, experiment_id):
  runs = pd.DataFrame(mlflow.search_runs(experiment_id))

  trained_lst = []
  not_trained_lst = []

  for i in range(runs.shape[0]):
    run = runs.iloc[i]
    params = {
        'vectorization' : run['params.vectorization'],
        'criterion' : run['params.criterion'],
        'optimizer' : run['params.optimizer'],
        'batch_size': int(run['params.batch_size']),
        'num_workers' : int(run['params.num_workers']),
        'lr' : float(run['params.lr']),
        'weight_decay' : float(run['params.weight_decay']),
        'num_epochs' : int(run['params.num_epochs']),
        'tolerance' : float(run['params.tolerance']),
        'patience' : int(run['params.patience'])
    }

    trained_lst.append(params)

  # check with combination
  for i in range(len(combinations)):
    already_trained = False

    for j in range(len(trained_lst)):
      if combinations[i] == trained_lst[j]:
        already_trained = True

    if not already_trained:
      not_trained_lst.append(combinations[i])

  return (not_trained_lst)

# remove already trained combinations from the list
not_trained_lst = remove_trained(all_combinations, experiment_id)

# check if any combinations are left to train

if len(not_trained_lst)>0:
  print(f"There are {len(not_trained_lst)} combinations left to train.")

else:
  print("All models have been trained.")

There are 11 combinations left to train.


In [36]:
# initialize an empty list to store the results
results = []

# record the start time for model training process
start_time = time.time()

# Loop throught each combinations of hyperparameters
for params in all_combinations:

  # Select the appropriate word vector based on vectorization parameter
  if params['vectorization']== 'cbow':
    matrix_train = matrix_train_cbow.copy()
    matrix_test = matrix_test_cbow.copy()
  elif params['vectorization']=='sg':
    matrix_train = matrix_train_sg.copy()
    matrix_test = matrix_test_sg.copy()

  # Train the neural network with the given parameters
  train_losses, test_losses, best_epoch = train_nn(matrix_train, matrix_test, params,
                                                   hidden_size = params.get('hidden_size',32),
                                                   input_size = params.get('input_size',300),
                                                   output_size = params.get('putput_size',1))

  # Append the training results to the results list
  results.append({
      'params':params,
      'train_losses':train_losses,
      'test_losses' : test_losses,
      'best_epoch' : best_epoch
  })

# record duration for training
duration = time.time()-start_time

print(f" Model training is completed. Elapsed time is {duration} secs")

print(results)


  0%|          | 0/500 [00:00<?, ?it/s]

Training neural network ... Epoch 0/499


  0%|          | 1/500 [00:08<1:09:58,  8.41s/it]

Training neural network ... Epoch 1/499
Best test loss epoch: 0


  0%|          | 2/500 [00:14<58:31,  7.05s/it]  

...

  5%|▌         | 27/500 [02:58<49:58,  6.34s/it]

Training neural network ... Epoch 27/499
Best test loss epoch: 13


  5%|▌         | 27/500 [03:06<54:21,  6.90s/it]

Early Stopping at epoch:  27
Neural network training has finished. Elasped time 186.18
Best model saved
Best test loss epoch: 13


Best model metrics: 
Accuracy: 1.0, Precision: 0.0, Recall: 0.0, F1 0.0
 Model training is completed. Elapsed time is 2618.5159730911255 secs
[{'params': {'vectorization': 'cbow', 'criterion': 'BCEWithLogitsLoss', 'optimizer': 'Adam', 'batch_size': 500, 'num_workers': 2, 'lr': 0.0001, 'weight_decay': 0.0001, 'num_epochs': 500, 'tolerance': 0.0003, 'patience': 25}, 'train_losses': [0.2151183, 0.040450133, 0.014181741, 0.0072901724, 0.0046219206, 0.0032908188, 0.0024854816, 0.0019338508, 0.0015121502, 0.0011751264, 0.00090230495, 0.0006821491, 0.00051099586, 0.00038317242, 0.00029007846, 0.00022423445, 0.00017885548, 0.0001471081, 0.00012637905, 0.00011247876, 0.00010345554, 9.7379816e-05, 9.347996e-05, 9.083494e-05, 8.9704095e-05, 8.932929e-05, 8.917594e-05, 8.920434e-05, 8.9051784e-05, 8.922096e-05, 8.900023e-05, 8.908585e-05, 8.915258e-05, 8.9072906e-05], 'test_losses': [0.07348335, 0.02101534, 0.009444037, 0.0055368394, 0.0037630512, 0.0027781688, 0.0021399304, 0.0016741301, 0.001307

## Mlflow Experiment Retrieval and Data Preview

In [39]:
# Function to retrieve experiment data from MLflow and format it into a DataFrame
def retrieve_experiments(id):
    # Fetch all runs for the given experiment ID and convert it to a DataFrame
    df = pd.DataFrame(mlflow.search_runs(experiment_ids=[id]))

    # Select and rename the columns of interest
    df = df[['run_id', 'params.vectorization', 'params.lr', 'params.tolerance', 'params.patience','params.weight_decay']]
    df = df.rename(columns={
        'run_id': 'Run ID', 'params.vectorization': 'Vectorization',
        'params.lr': 'LR', 'params.tolerance': 'Tolerance', 'params.patience': 'Patience', 'params.weight_decay': 'Weight Decay'})
    return df

# Retrieve and display the experiment data, sorted by 'Best Loss'
df_basic = retrieve_experiments(experiment_id)
display(df_basic.sort_values(by='Best Loss', ascending=True))

KeyError: 'Best Loss'